<a href="https://colab.research.google.com/github/KatrinaJMD/DAFT_0410/blob/main/Murder_Mystery_%5BPython%20SOLUTION%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STORY.**<br>
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​**murder**​ that occurred sometime on ​***January 15, 2018***​ and that it took place in ​**SQL City**​. Start by retrieving the corresponding crime scene report from the police department’s database.<br><br>

***Tip :***<br>
Here's a schema diagram to help you find your way around datasets and retrieve needed information.

![Schema Diagram](https://mystery.knightlab.com/schema.png)

**Find out who committed the murder.**

## ***PREPARE DATA***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

# KAGGLE
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Google Drive
import os
path = "/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/"
print("\nFiles inside Murder Mystery folder")
print("-" * 34)
csv_files = [file for file in os.listdir(path) if file.endswith(".csv")]
csv_files.sort()
for i, file in enumerate(csv_files):
    print(f"{i+1}. {file}")

pd.set_option('display.max_colwidth', None)
        
crime_scene_report = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/crime_scene_report.csv")
drivers_license = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/drivers_license.csv")
facebook_event_checkin = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/facebook_event_checkin.csv")
get_fit_now_check_in = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/get_fit_now_check_in.csv")
get_fit_now_member = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/get_fit_now_member.csv")
income = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/income.csv")
interview = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/interview.csv")
person = pd.read_csv("/content/drive/MyDrive/[01] Data Analytics - IronHack/[04] Extras/Murder Mystery Files/person.csv")

Mounted at /content/drive

Files inside Murder Mystery folder
----------------------------------
1. crime_scene_report.csv
2. drivers_license.csv
3. facebook_event_checkin.csv
4. get_fit_now_check_in.csv
5. get_fit_now_member.csv
6. income.csv
7. interview.csv
8. person.csv


## ***START***

### Retrieve corresponding crime scene report from police department's database

In [2]:
selected_crime = crime_scene_report[
                (crime_scene_report["date"] == 20180115) &
                (crime_scene_report["type"] == "murder") &
                (crime_scene_report["city"] == "SQL City")]
selected_crime

,date,type,description,city
1227,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


### Find persons who witnessed the crime

In [3]:
witness_1 = person[(person["address_street_name"] == "Northwestern Dr")].sort_values(by="address_number", ascending=False).head(1)
witness_2 = person[(person["name"].str.contains("Annabel", case=False)) & (person["address_street_name"] == "Franklin Ave")]

witnessess = pd.concat([witness_1, witness_2])
witnessess

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


### Get interview transcript from the 2 witnessess (Morty and Annabel)

In [4]:
witness_id = witnessess["id"].to_list()
witness_interview = interview[interview["person_id"].isin(witness_id)]
witness_interview

,person_id,transcript
4988,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
4989,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


### Get witnesses' person and interview data together

In [5]:
witness_person_interview = pd.merge(witnessess, witness_interview, how="outer", left_on="id", right_on="person_id")
witness_person_interview = witness_person_interview.drop(columns="person_id")
witness_person_interview

,id,name,license_id,address_number,address_street_name,ssn,transcript
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


### Retrieve suspect's gym info based on witnesses' interview

- ***witness 1:*** membership number on the bag started with "**48Z**"
- ***witness 2:*** working out last week on **January 9th**



In [6]:
gym_time = get_fit_now_check_in[(get_fit_now_check_in["membership_id"].str.contains("48Z", case=False)) &
                    (get_fit_now_check_in["check_in_date"].astype("str").str.contains("0109", case=False))]
gym_time

,membership_id,check_in_date,check_in_time,check_out_time
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


### Get suspect's gym membership info and verify the witness 1's description
- membership number on the bag started with "**48Z**"
- only **gold members** have those bags



In [7]:
suspect_gym_info = get_fit_now_member[get_fit_now_member["id"].isin(gym_time["membership_id"])]
suspect_gym_info

,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


### Check if suspect is recorded in person data

In [8]:
suspect_person = person[person["id"].isin(suspect_gym_info["person_id"])]
suspect_person

,id,name,license_id,address_number,address_street_name,ssn
2037,28819,Joe Germuska,173289,111,Fisk Rd,138909730
6327,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


### Check if suspect had any interviews

In [9]:
suspect_interview = interview[interview["person_id"].isin(suspect_gym_info["person_id"])]
suspect_interview

,person_id,transcript
4990,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


### Retrieve new suspect's info based on suspect's interview
- she has **red hair** and she drives a **Tesla Model S**
- she's around **5'5"** (65") or **5'7"** (67")

In [10]:
new_suspect_driverslicense = drivers_license[(drivers_license["hair_color"] == "red") &
                              (drivers_license["car_make"] == "Tesla") &
                              (drivers_license["height"].between(65, 68))]
new_suspect_driverslicense

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1105,202298,68,66,green,red,female,500123,Tesla,Model S
2054,291182,65,66,blue,red,female,08CM64,Tesla,Model S
9078,918773,48,65,black,red,female,917UU3,Tesla,Model S


### Get new suspect's name from person data using driver's license ID info

In [11]:
new_suspect_person = person[person["license_id"].isin(new_suspect_driverslicense["id"])]
new_suspect_person

,id,name,license_id,address_number,address_street_name,ssn
7664,78881,Red Korb,918773,107,Camerata Dr,961388910
8977,90700,Regina George,291182,332,Maple Ave,337169072
9985,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


### Check new suspect's activities based on suspect's interview
- she attended the **SQL Symphony Concert 3 times** in **December 2017**

In [12]:
symphony_concert = facebook_event_checkin[(facebook_event_checkin["event_name"].str.contains("SQL Symphony Concert", case=False)) &
                                          (facebook_event_checkin["date"].astype("str").str.contains("201712", case=False))]

### Filter activities where person_id is in new suspect's person data
- check if new suspect attended **SQL Symphony Concert 3 times** in **December 2017**


In [13]:
symphony_concert = symphony_concert[symphony_concert["person_id"].isin(new_suspect_person["id"])]
symphony_concert

,person_id,event_id,event_name,date
20006,99716,1143,SQL Symphony Concert,20171206
20007,99716,1143,SQL Symphony Concert,20171212
20008,99716,1143,SQL Symphony Concert,20171229


### Get the mastermind's name from the symphony concert data

In [14]:
mastermind = new_suspect_person[new_suspect_person["id"].isin(symphony_concert["person_id"])]
mastermind

,id,name,license_id,address_number,address_street_name,ssn
9985,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
